In [30]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import re
from selenium import webdriver
import os
from time import sleep

In [31]:
!which chromedriver

/usr/local/bin/chromedriver


In [32]:
# Open browser, scroll to bottom, get the HTML
browser = webdriver.Chrome()
url = ("https://health.usnews.com/best-hospitals/rankings/cancer")
browser.get(url)

# Scroll to bottom of page w/ a pause - this causes all hospitals to load
SCROLL_PAUSE_TIME = 4

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


html_source = browser.page_source
browser.quit()

# Create soup object using HTML source
soup = bs(html_source, "html5lib")
#print(soup)


In [33]:
# Scrape the soup object to get the hospital names and remove whitespaces
# and convert to upperscase

top50_list = []
           
for name in soup.find_all('h3', {'class' : 'block-flush heading-large'}):
    top50_list.append(name.get_text().strip().upper())
                      
top50_list

    

['UNIVERSITY OF TEXAS MD ANDERSON CANCER CENTER',
 'MEMORIAL SLOAN-KETTERING CANCER CENTER',
 'MAYO CLINIC',
 "DANA-FARBER/BRIGHAM AND WOMEN'S CANCER CENTER",
 'CLEVELAND CLINIC',
 'JOHNS HOPKINS HOSPITAL',
 'SEATTLE CANCER ALLIANCE/UNIVERSITY OF WASHINGTON MEDICAL CENTER',
 'H. LEE MOFFITT CANCER CENTER AND RESEARCH INSTITUTE',
 'UCSF MEDICAL CENTER',
 'HOSPITALS OF THE UNIVERSITY OF PENNSYLVANIA-PENN PRESBYTERIAN',
 'MAYO CLINIC-PHOENIX',
 'MASSACHUSETTS GENERAL HOSPITAL',
 'NORTHWESTERN MEMORIAL HOSPITAL',
 'STANFORD HEALTH CARE-STANFORD HOSPITAL',
 'SITEMAN CANCER CENTER',
 'UNIVERSITY OF MICHIGAN HOSPITALS-MICHIGAN MEDICINE',
 'USC NORRIS CANCER HOSPITAL-KECK MEDICAL CENTER OF USC',
 'UNIVERSITY OF IOWA HOSPITALS AND CLINICS',
 'WAKE FOREST BAPTIST MEDICAL CENTER',
 'OHIO STATE UNIVERSITY JAMES CANCER HOSPITAL',
 'CITY OF HOPE HELFORD CLINICAL RESEARCH HOSPITAL',
 'UCLA MEDICAL CENTER',
 'UPMC PRESBYTERIAN SHADYSIDE',
 'MUSC HEALTH-UNIVERSITY MEDICAL CENTER',
 'NEW YORK-PRESBYTERI

In [2]:
import pandas as pd
import requests
import json
import pprint
from pandas.io.json import json_normalize
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [3]:
# Base URL for the arcgis 
url = 'https://opendata.arcgis.com/datasets/a2817bf9632a43f5ad1c6b0c153b0fab_0.geojson'
data = requests.get(url)
data

<Response [200]>

In [7]:
hospitals_json = data.json()
hospitals_json['type']

'FeatureCollection'

In [37]:
#for hosp in top50_list:
top50_json = []
count = 0
for i in range(len(top50_list)): # tdqm allows loop tracking
    for feature in hospitals_json['features']:
        
        if fuzz.partial_ratio(feature['properties']['NAME'], top50_list[i]) > 95 \
            or fuzz.partial_ratio(feature['properties']['ALT_NAME'], top50_list[i]) > 95:
            #top50_json['features'].append(feature)
            print(feature['properties']['NAME'])
            top50_json.append(feature['properties'])
            count = count +1
        


UNIVERSITY OF TEXAS M.D. ANDERSON CANCER CENTER
MAYO CLINIC
MAYO CLINIC HOSPITAL METHODIST CAMPUS
MAYO CLINIC HEALTH SYS AUSTIN
MAYO CLINIC HEALTH SYSTEM IN WAYCROSS, INC
MAYO CLINIC HLTH SYSTM FRANCISCAN HLTHCARE SPARTA
MAYO CLINIC HEALTH SYSTEM - NORTHLAND IN BARRON
MAYO CLINIC HEALTH SYSTEM- CHIPPEWA VALLEY  INC
MAYO CLINIC HEALTH SYSTEM EAU CLAIRE HOSPITAL
MAYO CLINIC HLTH SYSTEM- FRANCISCAN MED CTR
MAYO CLINIC HEALTH SYSTEM - RED CEDAR INC
MAYO CLINIC HEALTH SYSTEM-OAKRIDGE INC
MAYO CLINIC HEALTH SYS ALBT LE
MAYO CLINIC HEALTH SYS CF
MAYO CLINIC HEALTH SYS FAIRMNT
MAYO CLINIC HEALTH SYS MANKATO
MAYO CLINIC HEALTH SYS ST JAME
MAYO CLINIC HEALTH SYSTEM IN RED WING
MAYO CLINIC HEALTH SYSTEM S F
MAYO CLINIC ARIZONA
MAYO CLINIC HOSPITAL ROCHESTER ST MARY'S CAMPUS
MAYO CLINIC HEALTH SYS WASECA
MAYO CLINIC HEALTH SYS CF
MAYO CLINIC HEALTH SYS L C
CLEVELAND CLINIC AVON HOSPITAL
CLEVELAND CLINIC REHABILITATION HOSPITAL, LLC
CLEVELAND CLINIC HOSPITAL
CLEVELAND CLINIC CHILDREN'S HOSPITAL FOR

In [104]:
zip = ''
name = 'USC'
addr = '1275 YORK AVENUE'
city = 'SALT LAKE CITY'
for feature in hospitals_json['features']:
    #if fuzz.partial_ratio(feature['properties']['NAME'], name) > 95:
    if fuzz.partial_ratio(feature['properties']['NAME'], name) >  95:
    #if fuzz.partial_ratio(feature['properties']['ADDRESS'], addr) > 95:
    #if fuzz.partial_ratio(feature['properties']['ZIP'], zip) == 100:
        pprint.pprint(feature['properties']['NAME'])

'NOLAND HOSPITAL TUSCALOOSA, LLC'
'VA MEDICAL CENTER - TUSCALOOSA'
'MUSCOGEE (CREEK) NATION LONG TERM ACUTE CARE HOSPITAL'
'MUSCOGEE (CREEK) NATION MEDICAL CENTER'
'LAC+USC MEDICAL CENTER'
'MUSCOGEE (CREEK) NATION PHYSICAL REHABILITATION CENTER'
'KECK HOSPITAL OF USC'
'USC KENNETH NORRIS, JR. CANCER HOSPITAL'
'USC VERDUGO HILLS HOSPITAL'
'TRINITY MUSCATINE'
'MUSC MEDICAL CENTER'


In [54]:
for item in top50_json:
    if item['NAME'] == 'UCSF MEDICAL CENTER AT MISSION BAY':
        pprint.pprint(item['ZIP'])
#len(top50_json)

'94158'


In [40]:
with open('hospitals.json', 'w') as outfile:
    json.dump(top50_json, outfile, sort_keys=True, indent=4)